# Предсказание стоимости жилья

В проекте нам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучим модель и сделаем предсказания на тестовой выборке. Для оценки качества модели будем использовать метрики RMSE, MAE и R2.

По условию задачи нам необходимо обучить две модели на разных наборах данных:
* используя все данные из файла;
* используя только числовые переменные, исключив категориальные.

## Подготовка данных

### Обзор данных 

In [2]:
# Инициализируем Spark-сессию
import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

# Импортируем необходимые функции для обучения моделей
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

RANDOM_SEED = 12112022

spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()

In [3]:
# Прочитаем датасет и выведем первые пять строк
df = spark.read.csv('/datasets/housing.csv', inferSchema=True, header=True)
df.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [4]:
# Выведем типы данных через метод PySpark
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Наш датасет состоит из 10 признаков. 9 признаков - вещественные числа, 1 признак - строковые значения.

In [5]:
# Посмотрим на общие сведения по датасету через метод info() из Pandas
df.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


В датасете чуть больше 20000 объектов. В признаке `total_bedrooms` присутствуют пропуски.

Известно, что каждая строка содержит агрегированную статистику о жилом массиве. Жилой массив — минимальная географическая единица с населением от 600 до 3000 человек в зависимости от штата.

`median_house_value` (медианная стоимость дома в жилом массиве) является целевым признаком.

В колонках датасета содержатся следующие данные:

* longitude — широта;
* latitude — долгота;
* housing_median_age — медианный возраст жителей жилого массива;
* total_rooms — общее количество комнат в домах жилого массива;
* total_bedrooms — общее количество спален в домах жилого массива;
* population — количество человек, которые проживают в жилом массиве;
* households — количество домовладений в жилом массиве;
* median_income — медианный доход жителей жилого массива;
* median_house_value — медианная стоимость дома в жилом массиве;
* ocean_proximity — близость к океану.

### Предобработка данных 

#### Удаление дубликатов


In [6]:
# Посчитаем количество строк в датасете
df.count()

20640

In [7]:
# Отбросим полные дубликаты, если они имеются, и выведем количество строк
df = df.drop_duplicates()
df.count()

20640

Количество строк не изменилось, следственно, полных дубликатов не обнаружилось.

#### Обработка пропусков

In [8]:
# Пропуски были обнаружены только в признаке total_bedrooms, посчитаем их количество.
df.select('total_bedrooms').where('total_bedrooms is Null').count()

207

С признаком `total_bedrooms` связан признак `total_rooms`. Логичным предположением является то, что с увеличением количества комнат будет увеличиватся количество спален. Взглянем на корреляцию этих двух признаков.

In [9]:
df.stat.corr('total_rooms', 'total_bedrooms')

0.9201961721166266

Сильная положительная корреляция, почти единица. В таком случае имеет смысл заполнить пропуски средней или медианной долей спален по отношению к общему количеству комнат в массивах. Для снижения влияния выбросов будем использовать медианное значение.

In [10]:
df = df.withColumn('bedrooms_ratio', F.round(F.col('total_bedrooms') / F.col('total_rooms'), 3))
df.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|bedrooms_ratio|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------+
|  -122.28|   37.81|              52.0|      340.0|          97.0|     200.0|      87.0|       1.5208|          112500.0|       NEAR BAY|         0.285|
|  -122.13|   37.67|              40.0|     1748.0|         318.0|     914.0|     317.0|       3.8676|          184000.0|       NEAR BAY|         0.182|
|  -122.07|   37.67|              27.0|     3239.0|         671.0|    1469.0|     616.0|       3.2465|          230600.0|       NEAR BAY|         0.207|
|  -122.13|   37.66|              19.0|      862.0|         167.0|     407.0|     

In [11]:
# Найдём медианное значение отношения кол-ва спален к кол-ву комнат
bedrooms_ratio_median = df.toPandas().bedrooms_ratio.median()
bedrooms_ratio_median

0.203

0.203 - таково медианное отношение кол-ва спален к кол-ву комнат. Теперь заполним пропуски с его помощью.

In [12]:
# Заполним пропуски по формуле "Количество комнат в массиве * медианное отношение кол-ва спален к кол-ву комнат"
df = df.withColumn('total_bedrooms', \
                   F.when(F.col('total_bedrooms').isNull(), F.round(F.col('total_rooms') * bedrooms_ratio_median, 0)) \
                   .otherwise(F.col('total_bedrooms')))

In [13]:
# Проверим, что у нас не осталось пропусков в признаке total_bedrooms
df.select('total_bedrooms').where('total_bedrooms is Null').count()

0

У нас остались пропуски в вычисляемом признаке `bedrooms_ratio`.

In [14]:
# Посчитаем их
df.select('bedrooms_ratio').where('bedrooms_ratio is Null').count()

207

In [15]:
# Отобразим первые пять объектов с пропусками в признаке bedrooms_ratio
df.where('bedrooms_ratio is Null').show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|bedrooms_ratio|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------+
|  -117.09|   32.77|              31.0|     3062.0|         622.0|    1263.0|     539.0|       3.0875|          291500.0|     NEAR OCEAN|          null|
|  -121.95|   38.03|               5.0|     5526.0|        1122.0|    3207.0|    1012.0|       4.0767|          143100.0|         INLAND|          null|
|  -119.69|   36.83|              32.0|     1098.0|         223.0|     726.0|     224.0|       1.4913|           54600.0|         INLAND|          null|
|  -118.18|   34.04|              42.0|     1670.0|         339.0|    1997.0|     

In [16]:
# Удалим колонку bedrooms_ratio для уменьшения корреляции с другими колонками
df = df.drop('bedrooms_ratio')
df.columns

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity']

#### Генерация признаков 

Добавим ещё 2 признака:

- Отношение количества комнат `total_rooms` к количеству домовладений `households`. Назовём колонку `rooms_per_household`.
- Отношение количества жителей `population` к количеству домовладений `households`. Назовём колонку `population_in_household`.

In [17]:
df = df.withColumn('rooms_per_household', F.col('total_rooms') / F.col('households'))
df = df.withColumn('population_in_household', F.col('population') / F.col('households'))

df.toPandas().head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,population_in_household
0,-122.28,37.81,52.0,340.0,97.0,200.0,87.0,1.5208,112500.0,NEAR BAY,3.908046,2.298851
1,-122.13,37.67,40.0,1748.0,318.0,914.0,317.0,3.8676,184000.0,NEAR BAY,5.514196,2.883281
2,-122.07,37.67,27.0,3239.0,671.0,1469.0,616.0,3.2465,230600.0,NEAR BAY,5.258117,2.384740
3,-122.13,37.66,19.0,862.0,167.0,407.0,183.0,4.3456,163000.0,NEAR BAY,4.710383,2.224044
4,-121.85,39.73,52.0,444.0,80.0,1107.0,98.0,3.4191,137500.0,INLAND,4.530612,11.295918


#### Описательная статистика


In [18]:
df.toPandas().describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,rooms_per_household,population_in_household
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.693120,1425.476744,499.539680,3.870671,206855.816909,5.429000,3.070655
std,2.003532,2.135952,12.585558,2181.615252,420.834094,1132.462122,382.329753,1.899822,115395.615874,2.474173,10.386050
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000,0.846154,0.692308
25%,-121.800000,33.930000,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000,4.440716,2.429741
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000,5.229129,2.818116
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000,6.052381,3.282261
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000,141.909091,1243.333333


In [19]:
df.toPandas().corr()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,rooms_per_household,population_in_household
longitude,1.000000,-0.924664,-0.108197,0.044568,0.068439,0.099773,0.055310,-0.015176,-0.045967,-0.027540,0.002476
latitude,-0.924664,1.000000,0.011173,-0.036100,-0.066125,-0.108785,-0.071035,-0.079809,-0.144160,0.106389,0.002366
housing_median_age,-0.108197,0.011173,1.000000,-0.361262,-0.321470,-0.296244,-0.302916,-0.119034,0.105623,-0.153277,0.013191
total_rooms,0.044568,-0.036100,-0.361262,1.000000,0.930862,0.857126,0.918484,0.198050,0.134153,0.133798,-0.024581
total_bedrooms,0.068439,-0.066125,-0.321470,0.930862,1.000000,0.877451,0.978731,-0.005675,0.051299,0.002551,-0.028319
population,0.099773,-0.108785,-0.296244,0.857126,0.877451,1.000000,0.907222,0.004834,-0.024650,-0.072213,0.069863
households,0.055310,-0.071035,-0.302916,0.918484,0.978731,0.907222,1.000000,0.013033,0.065843,-0.080598,-0.027309
median_income,-0.015176,-0.079809,-0.119034,0.198050,-0.005675,0.004834,0.013033,1.000000,0.688075,0.326895,0.018766
median_house_value,-0.045967,-0.144160,0.105623,0.134153,0.051299,-0.024650,0.065843,0.688075,1.000000,0.151948,-0.023737
rooms_per_household,-0.027540,0.106389,-0.153277,0.133798,0.002551,-0.072213,-0.080598,0.326895,0.151948,1.000000,-0.004852


In [20]:
df.toPandas().ocean_proximity.value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

In [21]:
df.toPandas()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,population_in_household
0,-122.28,37.81,52.0,340.0,97.0,200.0,87.0,1.5208,112500.0,NEAR BAY,3.908046,2.298851
1,-122.13,37.67,40.0,1748.0,318.0,914.0,317.0,3.8676,184000.0,NEAR BAY,5.514196,2.883281
2,-122.07,37.67,27.0,3239.0,671.0,1469.0,616.0,3.2465,230600.0,NEAR BAY,5.258117,2.384740
3,-122.13,37.66,19.0,862.0,167.0,407.0,183.0,4.3456,163000.0,NEAR BAY,4.710383,2.224044
4,-121.85,39.73,52.0,444.0,80.0,1107.0,98.0,3.4191,137500.0,INLAND,4.530612,11.295918
...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-120.89,37.59,33.0,1016.0,206.0,617.0,209.0,2.1510,195800.0,INLAND,4.861244,2.952153
20636,-121.62,39.09,21.0,2693.0,481.0,1337.0,435.0,3.8534,99700.0,INLAND,6.190805,3.073563
20637,-119.34,36.22,38.0,2708.0,460.0,1260.0,455.0,3.0905,78200.0,INLAND,5.951648,2.769231
20638,-119.18,34.28,17.0,4526.0,717.0,2088.0,655.0,5.6885,268200.0,NEAR OCEAN,6.909924,3.187786


Явных аномалий не замечено. Круглые значения скорее всего являются результатом небольшого количества домовладений.

Сильная корреляция между координатами - закономерно. Между количеством комнат, количеством спален, количеством домовладений, количеством населения так же имеется сильная корреляция. Это так же выглядит закономерным.

Присутствует средне-сильная корреляция между медианным доходом и медианной стоимостью жилья. Имеется слабо-средняя корреляция между медианным доходом и количеством комнат.

## Обучение моделей

### Подготовка признаков 

#### Подготовка категориальных признаков 

In [22]:
# Выделим категориальные признаки, числовые признаки и целевой признак
categorical_cols = ['ocean_proximity']
numerical_cols  = [
    'longitude', 
    'latitude', 
    'housing_median_age', 
    'total_rooms', 
    'total_bedrooms', 
    'households', 
    'median_income',
    'rooms_per_household', 
    'population_in_household'
]
target = 'median_house_value' 

In [23]:
# Трансформируем категориальный признак в числовое представление
indexer = StringIndexer(inputCols=categorical_cols, 
                        outputCols=[c+'_idx' for c in categorical_cols]) 
df = indexer.fit(df).transform(df)

cols = [c for c in df.columns for i in categorical_cols if (c.startswith(i))]
df.select(cols).show(10) 

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
|         INLAND|                1.0|
|         INLAND|                1.0|
|         INLAND|                1.0|
|         INLAND|                1.0|
|         INLAND|                1.0|
|         INLAND|                1.0|
+---------------+-------------------+
only showing top 10 rows



In [24]:
# Применим OHE-кодирование для трансформированного категориального признака
encoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                        outputCols=[c+'_ohe' for c in categorical_cols])
df = encoder.fit(df).transform(df)

cols = [c for c in df.columns for i in categorical_cols if (c.startswith(i))]
df.select(cols).show(10)

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|         INLAND|                1.0|      (4,[1],[1.0])|
|         INLAND|                1.0|      (4,[1],[1.0])|
|         INLAND|                1.0|      (4,[1],[1.0])|
|         INLAND|                1.0|      (4,[1],[1.0])|
|         INLAND|                1.0|      (4,[1],[1.0])|
|         INLAND|                1.0|      (4,[1],[1.0])|
+---------------+-------------------+-------------------+
only showing top 10 rows



In [25]:
# Трансформируем категориальный признак в вектор для обучения
categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols], 
                        outputCol="categorical_features")
df = categorical_assembler.transform(df)

#### Подготовка числовых признаков


In [26]:
# Соберём числовые признаки в единый вектор
numerical_assembler = \
        VectorAssembler(inputCols=numerical_cols, 
                        outputCol="numerical_features")
df = numerical_assembler.transform(df)

In [27]:
# Отмасштабируем числовые признаки
standardScaler = \
        StandardScaler(inputCol='numerical_features', 
                       outputCol="numerical_features_scaled")
df = standardScaler.fit(df).transform(df)

#### Объединение признаков в вектор

In [28]:
# Посмотрим на все получившиеся признаки вместе
df.columns

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity',
 'rooms_per_household',
 'population_in_household',
 'ocean_proximity_idx',
 'ocean_proximity_ohe',
 'categorical_features',
 'numerical_features',
 'numerical_features_scaled']

In [29]:
# Соберем все трансформированные признаки в единый вектор
all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol='features') 
df = final_assembler.transform(df)

df.select('features').show(10)

+--------------------+
|            features|
+--------------------+
|[0.0,0.0,0.0,1.0,...|
|[0.0,0.0,0.0,1.0,...|
|[0.0,0.0,0.0,1.0,...|
|[0.0,0.0,0.0,1.0,...|
|[0.0,1.0,0.0,0.0,...|
|[0.0,1.0,0.0,0.0,...|
|[0.0,1.0,0.0,0.0,...|
|[0.0,1.0,0.0,0.0,...|
|[0.0,1.0,0.0,0.0,...|
|[0.0,1.0,0.0,0.0,...|
+--------------------+
only showing top 10 rows



#### Разделение на выборки

In [30]:
train_data, test_data = df.randomSplit([.8,.2], seed=RANDOM_SEED)
display(train_data.count(), test_data.count())

16530

4110

### Обучение моделей 

По условию задачи нам необходимо обучить две модели на разных наборах данных:
* используя все данные из файла;
* используя только числовые переменные, исключив категориальные.

#### Обучение модели с использованием всех данных

In [31]:
# Обучим модель линейной регрессии
lr = LinearRegression(labelCol=target, featuresCol='features')

model = lr.fit(train_data)

22/11/17 09:33:57 WARN Instrumentation: [21bd75f2] regParam is zero, which might cause numerical instability and overfitting.
22/11/17 09:33:57 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/11/17 09:33:57 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/11/17 09:34:05 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/11/17 09:34:05 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [32]:
# Сделаем предсказания целевого признака
predictions = model.transform(test_data)

predictedLabes = predictions.select(target, 'prediction')
predictedLabes.show(10)

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          252000.0|266741.85915008746|
|          500001.0|439235.99356851866|
|          413000.0|353556.21564060496|
|          112500.0| 158760.7525684801|
|          192800.0| 282057.1848999425|
|          219300.0|161735.58523438824|
|          141800.0|120272.91092541255|
|           62000.0|    90767.00788503|
|          500001.0|349106.61427363986|
|          450000.0|334976.32641096134|
+------------------+------------------+
only showing top 10 rows



In [33]:
# Оценим качество предсказаний по метрикам RMSE, MAE, R2
rmse = RegressionEvaluator(labelCol=target,
                           metricName='rmse').evaluate(predictions)

mae = RegressionEvaluator(labelCol=target,
                          metricName='mae').evaluate(predictions)

r2 = RegressionEvaluator(labelCol=target,
                         metricName='r2').evaluate(predictions)

display(f'RMSE: {rmse}')
display(f'MAE: {mae}')
display(f'R2: {r2}')

'RMSE: 69317.80225900839'

'MAE: 51075.170279426566'

'R2: 0.6396752541788671'

#### Обучение модели с использованием только числовых данных 

In [34]:
# Обучим модель линейной регрессии
lr = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled')

model = lr.fit(train_data)

22/11/17 09:34:35 WARN Instrumentation: [a153bf63] regParam is zero, which might cause numerical instability and overfitting.


In [35]:
# Сделаем предсказания целевого признака
predictions = model.transform(test_data)

predictedLabes = predictions.select(target, 'prediction')
predictedLabes.show(10)

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          252000.0| 269033.9868632681|
|          500001.0| 443960.3291619164|
|          413000.0|345305.71643787064|
|          112500.0|154325.97761280555|
|          192800.0|  279103.250279177|
|          219300.0| 166562.0992065766|
|          141800.0|133650.75990496855|
|           62000.0|108658.22400977928|
|          500001.0|351436.53931543557|
|          450000.0|198283.75856643263|
+------------------+------------------+
only showing top 10 rows



In [36]:
# Оценим качество предсказаний по метрикам RMSE, MAE, R2
rmse = RegressionEvaluator(labelCol=target,
                           metricName='rmse').evaluate(predictions)

mae = RegressionEvaluator(labelCol=target,
                          metricName='mae').evaluate(predictions)

r2 = RegressionEvaluator(labelCol=target,
                         metricName='r2').evaluate(predictions)

display(f'RMSE: {rmse}')
display(f'MAE: {mae}')
display(f'R2: {r2}')

'RMSE: 69988.05441699916'

'MAE: 52150.84888473936'

'R2: 0.6326734152564195'

## Анализ результатов

Мы подготовили и обучили 2 модели линейной регрессии для прогноза медианной стоимости дома в жилом массиве. Получены следующие значения контрольных метрик:

|Метрика|Обучение на всех данных| Обучение без категориального признака|
|---|---|---|
|RMSE|69317.80225900839|69988.05441699916|
|MAE|51075.170279426566|52150.84888473936|
|R2|0.6396752541788671|0.6326734152564195|

По представленной таблице можно заметить, что обучение без использования категориального признака дало результат чуть хуже, чем обучение на полном наборе данных.